In [1]:
#!gcloud auth application-default login

In [2]:
#!gcloud auth login

In [ ]:
import os

import hail as hl
import pyspark.sql.functions as f

from gentropy.common.session import Session
from gentropy.dataset.study_locus import StudyLocus

hail_dir = os.path.dirname(hl.__file__)
session = Session(hail_home=hail_dir, start_hail=True, extended_spark_conf={
    "spark.driver.memory": "12g","spark.kryoserializer.buffer.max": "500m","spark.driver.maxResultSize":"2g",
    "spark.hadoop.fs.gs.requester.pays.buckets": "requester-pays-bucket1,requester-pays-bucket2",
    "spark.hadoop.fs.gs.requester.pays.project.id": "open-targets-genetics-dev",
    "spark.hadoop.fs.gs.requester.pays.mode":"AUTO"})

Loading BokehJS ...

24/11/23 15:59:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.3.4
SparkUI available at http://mib118093s:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.127-bb535cd096c5
LOGGING: writing to /dev/null


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54109)
Traceback (most recent call last):
  File "/Users/yt4/.pyenv/versions/3.10.8/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/yt4/.pyenv/versions/3.10.8/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/yt4/.pyenv/versions/3.10.8/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/yt4/.pyenv/versions/3.10.8/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/yt4/Library/Caches/pypoetry/virtualenvs/gentropy-krNFZEZg-py3.10/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/yt4/Library/Caches/pypoetry/virtualenvs/gentropy-krNFZEZg-py3.1

In [4]:
fm=session.spark.read.parquet("gs://ot_orchestration/releases/24.11_freeze10/locus_to_gene_feature_matrix")
fm10=fm.select("studyLocusId").distinct().cache()
fm10.count()


486888

In [5]:
fm=session.spark.read.parquet("gs://ot_orchestration/releases/24.10_freeze6/locus_to_gene_feature_matrix")
fm6=fm.select("studyLocusId").distinct().cache()
fm6.count()


475029

In [6]:
def validate_pred(
    valid_set,
    prediction_set,
    score_column,
    gsp_colum,
    positive,
    score_thr,
):
    valid_set=valid_set.join(prediction_set, on=["studyLocusId","geneId"], how="left").cache()
    valid_set=valid_set.fillna(0)
    df=(valid_set.select(score_column,gsp_colum)
        .withColumn("gsp_status", f.when(f.col(gsp_colum)==positive,1).otherwise(0))
        .withColumn(
        "score_to_use",
        f.when(f.col(score_column) >= score_thr, 1).otherwise(0)
        )
        .drop(gsp_colum,score_column)
        .cache())

    counts_df = df.groupBy("gsp_status", "score_to_use").agg(f.count("*").alias("count"))
    counts = { (row.gsp_status, row.score_to_use): row["count"] for row in counts_df.collect() }
    count_1_1 = counts.get((1, 1), 0)
    count_1_0 = counts.get((1, 0), 0)
    count_0_1 = counts.get((0, 1), 0)
    count_0_0 = counts.get((0, 0), 0)
    TP=count_1_1
    FP=count_0_1
    TN=count_0_0
    FN=count_1_0
    # Print the results
    print(f"Prec: {round(TP/(TP+FP),2)}")
    print(f"Sens: {round(TP/(TP+FN),2)}")
    print(f"Sele: {round(TN/(FP+TN),2)}")
    print(f"Count P: {TP+FN}")
    print(f"Count N: {TN+FP}")

In [7]:
old=session.spark.read.parquet("gs://genetics-portal-dev-staging/l2g/220712/predictions/l2g.full.220712.parquet")
old=old.filter(f.col("y_proba_full_model")>=0.05).select("study_id","gene_id","y_proba_full_model")
old=old.withColumnRenamed("study_id","studyId").withColumnRenamed("gene_id","geneId").cache()
old.count()

781806

In [8]:
def MAIN(filtered_l2g,fm_to_filter,thr=0.5):

    filtered_df = filtered_l2g.filter(f.col("score") >= thr).select("studyLocusId")

    # Step 2: Group by studyLocusId and count occurrences
    grouped_df = filtered_df.groupBy("studyLocusId").count()

    grouped_df=grouped_df.withColumnRenamed("count","count_studyLocusId")
    # Step 3: Aggregate the counts
    result_df = grouped_df.groupBy("count_studyLocusId").count().cache()
    # Show the results
    #result_df.show(truncate=False)

    # Calculate the number of instances
    only_once = result_df.filter(f.col("count_studyLocusId") == 1).select(f.sum("count")).collect()[0][0]
    twice = result_df.filter(f.col("count_studyLocusId") == 2).select(f.sum("count")).collect()[0][0]
    more_than_twice = result_df.filter(f.col("count_studyLocusId") > 2).select(f.sum("count")).collect()[0][0]

    # Print the results
    print(f"1 per cs: {only_once}")
    print(f"2 per cs: {twice}")
    print(f">2 per cs: {more_than_twice}")

    # OLD VS NEW

    new=filtered_l2g.filter(f.col("score")>=0.05).select("studyLocusId","geneId","score")
    cs=StudyLocus.from_parquet(session,"gs://ot_orchestration/releases/24.11_freeze10/credible_set")
    cs=cs.df.select("studyLocusId","studyId")
    result_df = new.join(cs, on="studyLocusId", how="left")

    result_df=result_df.join(old,["studyId","geneId"],"inner")
    correlation = result_df.stat.corr("score", "y_proba_full_model")
    print(f"Correlation between score and y_proba_full_model: {correlation}")

    # INSAMPLE VALIDATION
    print("")
    print("")
    print("training_v5_sum_col_string_filtered")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_sum_col_string_filtered.json"
    valid_set=session.spark.read.json(path)
    valid_set=valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=valid_set.count()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr
    )

    print("")
    print("")
    # insample validation
    print("training_v5_1_sum_col_string_filtered")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_1_sum_col_string_filtered.tsv"
    valid_set=session.spark.read.csv(path,sep="\t",header=True)
    valid_set_to_remove=valid_set.cache()
    valid_set=valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=valid_set.count()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="GSP",
        positive=1,
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 1
    print("training_v5_1_sum_col_string_filtered_full")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_1_sum_col_string_filtered_full.tsv"
    valid_set=session.spark.read.csv(path,sep="\t",header=True)
    valid_set=valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=valid_set.count()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="GSP",
        positive=1,
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 1
    print("training_v5_1_sum_col_string_filtered_full - 5.1_train")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_1_sum_col_string_filtered_full.tsv"
    valid_set=session.spark.read.csv(path,sep="\t",header=True)
    filtered_valid_set = valid_set.join(
        valid_set_to_remove.select("studyLocusId", "geneId"),
        on=["studyLocusId", "geneId"],
        how="left_anti"
    )
    filtered_valid_set=filtered_valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=filtered_valid_set.count()
    validate_pred(
        valid_set=filtered_valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="GSP",
        positive=1,
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 2
    print("featureMatrix_12_72e1caafae1514639eeb")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/featureMatrix_12_72e1caafae1514639eeb.table.json"
    valid_set=session.spark.read.json(path)
    columns = valid_set.select("columns").first()[0]
    data = valid_set.select("data").first()[0]
    new_df = session.spark.createDataFrame(data, schema=columns)
    new_df=new_df.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=new_df.count()
    validate_pred(
        valid_set=new_df,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 4
    print("featureMatrix_12_72e1caafae1514639eeb -5.1_train")
    filtered_valid_set = new_df.join(
        valid_set_to_remove.select("studyLocusId", "geneId"),
        on=["studyLocusId", "geneId"],
        how="left_anti"
    )
    filtered_valid_set=filtered_valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=filtered_valid_set.count()
    validate_pred(
        valid_set=filtered_valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 5
    print("EGL_training_set_2410_freeze7_stringent_distribution_unbiased")
    path="gs://genetics-portal-dev-analysis/xg1/Chembl_l2g_goldstandards/EGL_training_set_2410_freeze7_stringent_distribution_unbiased.parquet"
    valid_set=session.spark.read.parquet(path)
    valid_set=valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=valid_set.count()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="Goldstandard_positive",
        positive=1,
        score_thr=thr,
    )

    print("")
    print("")
    print("training_v10_v6_sum_col_all_cs_string")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v10_v6_sum_col_all_cs_string.json"
    valid_set=session.spark.read.json(path)
    valid_set=valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=valid_set.count()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )

    print("")
    print("")
    print("training_v10_v6_sum_col_best_cs_selected_string")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v10_v6_sum_col_best_cs_selected_string.json"
    valid_set=session.spark.read.json(path)
    valid_set=valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=valid_set.count()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )

    print("")
    print("")
    print("training_v10_1_full_string_filtered")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v10_1_full_string_filtered.json"
    valid_set=session.spark.read.json(path)
    valid_set=valid_set.join(fm_to_filter, on="studyLocusId", how="inner").cache()
    tmp=valid_set.count()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )


In [9]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v5/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 287451
2 per cs: 29301
>2 per cs: 2308


Correlation between score and y_proba_full_model: 0.4777363129598357


training_v5_sum_col_string_filtered


Prec: 0.96
Sens: 0.71
Sele: 1.0
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.96
Sens: 0.73
Sele: 1.0
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.79
Sens: 0.55
Sele: 0.99
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.84
Sens: 0.55
Sele: 0.98
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.57
Sens: 0.4
Sele: 0.93
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.78
Sens: 0.56
Sele: 0.99
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.96
Sens: 0.75
Sele: 1.0
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.74
Sens: 0.53
Sele: 0.99
Count P: 8013
Count N: 111846


In [10]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v5.1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 299915
2 per cs: 29239
>2 per cs: 4209


Correlation between score and y_proba_full_model: 0.478351225259997


training_v5_sum_col_string_filtered


Prec: 0.89
Sens: 0.73
Sele: 0.99
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.93
Sens: 0.76
Sele: 1.0
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.74
Sens: 0.57
Sele: 0.99
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.81
Sens: 0.54
Sele: 0.97
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.42
Sele: 0.93
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.74
Sens: 0.59
Sele: 0.98
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.91
Sens: 0.78
Sele: 0.99
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.7
Sens: 0.56
Sele: 0.98
Count P: 8013
Count N: 111846


In [11]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v4/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 240951
2 per cs: 12794
>2 per cs: 1014


Correlation between score and y_proba_full_model: 0.43353720782902355


training_v5_sum_col_string_filtered


Prec: 0.82
Sens: 0.45
Sele: 0.99
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.83
Sens: 0.47
Sele: 0.99
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.95
Sens: 0.7
Sele: 1.0
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.88
Sens: 0.59
Sele: 0.98
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.3
Sele: 0.95
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.95
Sens: 0.72
Sele: 1.0
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.85
Sens: 0.52
Sele: 0.99
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.93
Sens: 0.71
Sele: 1.0
Count P: 8013
Count N: 111846


In [12]:
l2g=session.spark.read.parquet("gs://ot-team/irene/l2g/13112024/locus_to_gene_predictions_0")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 283231
2 per cs: 86768
>2 per cs: 17696


Correlation between score and y_proba_full_model: 0.46694813068198104


training_v5_sum_col_string_filtered


Prec: 0.55
Sens: 0.53
Sele: 0.97
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.57
Sens: 0.55
Sele: 0.97
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.6
Sens: 0.63
Sele: 0.97
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.95
Sens: 0.91
Sele: 0.99
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.37
Sens: 0.37
Sele: 0.86
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.59
Sens: 0.63
Sele: 0.97
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.56
Sens: 0.56
Sele: 0.97
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.56
Sens: 0.61
Sele: 0.97
Count P: 8013
Count N: 111846


In [13]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 240906
2 per cs: 17836
>2 per cs: 1908


Correlation between score and y_proba_full_model: 0.4406493404617708


training_v5_sum_col_string_filtered


Prec: 0.91
Sens: 0.59
Sele: 1.0
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.95
Sens: 0.63
Sele: 1.0
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.79
Sens: 0.46
Sele: 0.99
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.87
Sens: 0.43
Sele: 0.99
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.7
Sens: 0.36
Sele: 0.96
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.78
Sens: 0.49
Sele: 0.99
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.93
Sens: 0.64
Sele: 1.0
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.75
Sens: 0.47
Sele: 0.99
Count P: 8013
Count N: 111846


In [14]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm1/v1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 254898
2 per cs: 15209
>2 per cs: 1660


Correlation between score and y_proba_full_model: 0.43092663945203963


training_v5_sum_col_string_filtered


Prec: 0.9
Sens: 0.6
Sele: 0.99
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.92
Sens: 0.64
Sele: 1.0
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.76
Sens: 0.48
Sele: 0.99
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.83
Sens: 0.47
Sele: 0.98
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.6
Sens: 0.36
Sele: 0.95
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.76
Sens: 0.51
Sele: 0.99
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.93
Sens: 0.67
Sele: 1.0
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.72
Sens: 0.48
Sele: 0.99
Count P: 8013
Count N: 111846


In [15]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v6/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 39252
2 per cs: 38972
>2 per cs: 361762


Correlation between score and y_proba_full_model: -0.04743108564677633


training_v5_sum_col_string_filtered


Prec: 0.05
Sens: 0.51
Sele: 0.27
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.05
Sens: 0.51
Sele: 0.26
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.04
Sens: 0.39
Sele: 0.25
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.06
Sens: 0.23
Sele: 0.25
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.3
Sens: 0.61
Sele: 0.68
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.04
Sens: 0.45
Sele: 0.27
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.07
Sens: 0.63
Sele: 0.3
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.05
Sens: 0.49
Sele: 0.28
Count P: 8013
Count N: 111846


In [16]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v7/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 41473
2 per cs: 42256
>2 per cs: 347341


Correlation between score and y_proba_full_model: -0.027994214254160613


training_v5_sum_col_string_filtered


Prec: 0.05
Sens: 0.46
Sele: 0.31
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.05
Sens: 0.47
Sele: 0.31
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.04
Sens: 0.36
Sele: 0.3
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.05
Sens: 0.16
Sele: 0.3
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.33
Sens: 0.56
Sele: 0.74
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.04
Sens: 0.44
Sele: 0.31
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.07
Sens: 0.6
Sele: 0.33
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.05
Sens: 0.48
Sele: 0.32
Count P: 8013
Count N: 111846


In [17]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm1/v5/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 286370
2 per cs: 28910
>2 per cs: 3418


Correlation between score and y_proba_full_model: 0.4865296938984367


training_v5_sum_col_string_filtered


Prec: 0.94
Sens: 0.71
Sele: 1.0
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.93
Sens: 0.72
Sele: 1.0
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.76
Sens: 0.54
Sele: 0.99
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.86
Sens: 0.55
Sele: 0.98
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.39
Sele: 0.93
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.75
Sens: 0.56
Sele: 0.99
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.91
Sens: 0.74
Sele: 0.99
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.71
Sens: 0.53
Sele: 0.98
Count P: 8013
Count N: 111846


In [18]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v8/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 250753
2 per cs: 14485
>2 per cs: 2441


Correlation between score and y_proba_full_model: 0.46397662553726826


training_v5_sum_col_string_filtered


Prec: 0.84
Sens: 0.47
Sele: 0.99
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.86
Sens: 0.49
Sele: 0.99
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.94
Sens: 0.7
Sele: 1.0
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.87
Sens: 0.58
Sele: 0.98
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.6
Sens: 0.33
Sele: 0.95
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.95
Sens: 0.72
Sele: 1.0
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.89
Sens: 0.55
Sele: 0.99
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.92
Sens: 0.7
Sele: 1.0
Count P: 8013
Count N: 111846


In [19]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm1/v5.1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 310153
2 per cs: 27580
>2 per cs: 3077


Correlation between score and y_proba_full_model: 0.4992247830440022


training_v5_sum_col_string_filtered


Prec: 0.9
Sens: 0.71
Sele: 0.99
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.93
Sens: 0.75
Sele: 1.0
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.75
Sens: 0.61
Sele: 0.99
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.84
Sens: 0.61
Sele: 0.97
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.43
Sele: 0.93
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.75
Sens: 0.62
Sele: 0.98
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.93
Sens: 0.77
Sele: 1.0
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.71
Sens: 0.59
Sele: 0.98
Count P: 8013
Count N: 111846


In [20]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/v5.1_best_cv/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm6,thr=0.5)

1 per cs: 345498
2 per cs: 18619
>2 per cs: 2227


Correlation between score and y_proba_full_model: 0.5299767364218452


training_v5_sum_col_string_filtered


Prec: 0.71
Sens: 0.52
Sele: 0.98
Count P: 538
Count N: 6883


training_v5_1_sum_col_string_filtered


Prec: 0.73
Sens: 0.55
Sele: 0.98
Count P: 488
Count N: 6111


training_v5_1_sum_col_string_filtered_full


Prec: 0.82
Sens: 0.7
Sele: 0.99
Count P: 10598
Count N: 145702


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.78
Sens: 0.6
Sele: 0.96
Count P: 375
Count N: 1674


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.48
Sens: 0.35
Sele: 0.92
Count P: 905
Count N: 4083


training_v10_v6_sum_col_all_cs_string


Prec: 0.82
Sens: 0.72
Sele: 0.99
Count P: 7943
Count N: 110027


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.75
Sens: 0.6
Sele: 0.98
Count P: 306
Count N: 3879


training_v10_1_full_string_filtered


Prec: 0.78
Sens: 0.7
Sele: 0.99
Count P: 8013
Count N: 111846


In [21]:
l2g=session.spark.read.parquet("gs://ot_orchestration/releases/24.11_freeze10/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm10,thr=0.5)

1 per cs: 346217
2 per cs: 16801
>2 per cs: 2075


Correlation between score and y_proba_full_model: 0.5699437332506234


training_v5_sum_col_string_filtered


Prec: 0.73
Sens: 0.54
Sele: 0.98
Count P: 351
Count N: 4506


training_v5_1_sum_col_string_filtered


Prec: 0.76
Sens: 0.58
Sele: 0.99
Count P: 313
Count N: 3985


training_v5_1_sum_col_string_filtered_full


Prec: 0.82
Sens: 0.7
Sele: 0.99
Count P: 7949
Count N: 110018


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.79
Sens: 0.59
Sele: 0.94
Count P: 58
Count N: 150


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.65
Sens: 0.43
Sele: 0.95
Count P: 449
Count N: 2144


training_v10_v6_sum_col_all_cs_string


Prec: 0.83
Sens: 0.69
Sele: 0.99
Count P: 10876
Count N: 149641


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.76
Sens: 0.53
Sele: 0.99
Count P: 498
Count N: 6354


training_v10_1_full_string_filtered


Prec: 0.8
Sens: 0.68
Sele: 0.99
Count P: 10549
Count N: 145299


In [22]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/v5.1_best_cv_freeze10/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm10,thr=0.5)

1 per cs: 318745
2 per cs: 14470
>2 per cs: 1455


Correlation between score and y_proba_full_model: 0.5463524504950565


training_v5_sum_col_string_filtered


Prec: 0.76
Sens: 0.48
Sele: 0.99
Count P: 351
Count N: 4506


training_v5_1_sum_col_string_filtered


Prec: 0.77
Sens: 0.51
Sele: 0.99
Count P: 313
Count N: 3985


training_v5_1_sum_col_string_filtered_full


Prec: 0.84
Sens: 0.64
Sele: 0.99
Count P: 7949
Count N: 110018


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.79
Sens: 0.57
Sele: 0.94
Count P: 58
Count N: 150


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.64
Sens: 0.39
Sele: 0.96
Count P: 449
Count N: 2144


training_v10_v6_sum_col_all_cs_string


Prec: 0.84
Sens: 0.63
Sele: 0.99
Count P: 10876
Count N: 149641


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.76
Sens: 0.47
Sele: 0.99
Count P: 498
Count N: 6354


training_v10_1_full_string_filtered


Prec: 0.83
Sens: 0.63
Sele: 0.99
Count P: 10549
Count N: 145299


In [23]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/v5.1_best_cv_freeze10_v3/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm10,thr=0.5)

1 per cs: 297538
2 per cs: 15904
>2 per cs: 1350


Correlation between score and y_proba_full_model: 0.5533307253398303


training_v5_sum_col_string_filtered


Prec: 0.71
Sens: 0.49
Sele: 0.98
Count P: 351
Count N: 4506


training_v5_1_sum_col_string_filtered


Prec: 0.73
Sens: 0.52
Sele: 0.98
Count P: 313
Count N: 3985


training_v5_1_sum_col_string_filtered_full


Prec: 0.87
Sens: 0.66
Sele: 0.99
Count P: 7949
Count N: 110018


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.81
Sens: 0.5
Sele: 0.95
Count P: 58
Count N: 150


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.65
Sens: 0.4
Sele: 0.95
Count P: 449
Count N: 2144


training_v10_v6_sum_col_all_cs_string


Prec: 0.86
Sens: 0.65
Sele: 0.99
Count P: 10876
Count N: 149641


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.71
Sens: 0.47
Sele: 0.99
Count P: 498
Count N: 6354


training_v10_1_full_string_filtered


Prec: 0.86
Sens: 0.65
Sele: 0.99
Count P: 10549
Count N: 145299


In [24]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/training_v10_v6_sum_col_best_cs_selected_string/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm10,thr=0.5)

1 per cs: 311448
2 per cs: 21199
>2 per cs: 1338


Correlation between score and y_proba_full_model: 0.5838065729445086


training_v5_sum_col_string_filtered


Prec: 0.84
Sens: 0.62
Sele: 0.99
Count P: 351
Count N: 4506


training_v5_1_sum_col_string_filtered


Prec: 0.87
Sens: 0.66
Sele: 0.99
Count P: 313
Count N: 3985


training_v5_1_sum_col_string_filtered_full


Prec: 0.76
Sens: 0.57
Sele: 0.99
Count P: 7949
Count N: 110018


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.82
Sens: 0.57
Sele: 0.95
Count P: 58
Count N: 150


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.7
Sens: 0.46
Sele: 0.96
Count P: 449
Count N: 2144


training_v10_v6_sum_col_all_cs_string


Prec: 0.77
Sens: 0.55
Sele: 0.99
Count P: 10876
Count N: 149641


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.88
Sens: 0.6
Sele: 0.99
Count P: 498
Count N: 6354


training_v10_1_full_string_filtered


Prec: 0.74
Sens: 0.54
Sele: 0.99
Count P: 10549
Count N: 145299


In [25]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/training_v10_v6_sum_col_all_cs_string/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g,fm_to_filter=fm10,thr=0.5)

1 per cs: 350900
2 per cs: 16121
>2 per cs: 1310


Correlation between score and y_proba_full_model: 0.6060657332512656


training_v5_sum_col_string_filtered


Prec: 0.73
Sens: 0.54
Sele: 0.98
Count P: 351
Count N: 4506


training_v5_1_sum_col_string_filtered


Prec: 0.76
Sens: 0.57
Sele: 0.99
Count P: 313
Count N: 3985


training_v5_1_sum_col_string_filtered_full


Prec: 0.83
Sens: 0.69
Sele: 0.99
Count P: 7949
Count N: 110018


training_v5_1_sum_col_string_filtered_full - 5.1_train


...

Prec: 0.8
Sens: 0.55
Sele: 0.95
Count P: 58
Count N: 150


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.63
Sens: 0.42
Sele: 0.95
Count P: 449
Count N: 2144


training_v10_v6_sum_col_all_cs_string


Prec: 0.83
Sens: 0.68
Sele: 0.99
Count P: 10876
Count N: 149641


training_v10_v6_sum_col_best_cs_selected_string


Prec: 0.75
Sens: 0.51
Sele: 0.99
Count P: 498
Count N: 6354


training_v10_1_full_string_filtered


Prec: 0.8
Sens: 0.67
Sele: 0.99
Count P: 10549
Count N: 145299
